<a href="https://colab.research.google.com/github/tankien76/CS114.K21/blob/master/Sarcasm_headlines.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mô tả bài toán
- Input: Các tiêu đề (headlines)
- Output: Tiêu đề châm biếm hay không châm biếm

Code tham khảo: https://towardsdatascience.com/sarcasm-detection-step-towards-sentiment-analysis-84cb013bb6db
                

Code tham khảo: https://pythonspot.com/nltk-stop-words/

# Prepare dataset

Thu thập các đường link của bài báo(article_link), tiêu đề sarcasm(is_sarcastic=1) từ trang https://www.theonion.com và not sarcasm(is_sarcastic=0) từ trang https://www.huffingtonpost.com

Upload file kaggle.json để lấy dữ liệu từ kaggle

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving kaggle.json to kaggle.json


Tải dữ liệu về từ kaggle và giải nén để lấy dataset  (Sarcasm_Headlines_Dataset_v2.json)

In [ ]:
!ls -lha kaggle.json
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

-rw-r--r-- 1 root root 68 Jun 25 10:44 kaggle.json


In [ ]:
!kaggle datasets download -d rmisra/news-headlines-dataset-for-sarcasm-detection

  0% 0.00/3.30M [00:00<?, ?B/s]
100% 3.30M/3.30M [00:00<00:00, 110MB/s]


In [ ]:
!unzip news-headlines-dataset-for-sarcasm-detection.zip

Archive:  news-headlines-dataset-for-sarcasm-detection.zip
  inflating: Sarcasm_Headlines_Dataset.json  
  inflating: Sarcasm_Headlines_Dataset_v2.json  


# Feature engineering

download punkt trong thư viện nltk để tách từ khi xử lý dữ liệu

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

download stopwords trong thư viện nltk để xử lý các từ không cần thiết trong tiếng anh

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

Import các thư viện sử dụng:
- pandas : đọc dữ liệu file json
- numpy : xử lý dữ liệu text
- nltk : xử lý ngôn ngữ tự nhiên 

In [ ]:
import pandas as pd, numpy as np, re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

Đọc dữ liệu từ file Sarcasm_Headlines_Dataset_v2.json

In [ ]:
# Loading data from json file
data = pd.read_json("Sarcasm_Headlines_Dataset_v2.json", lines = True)
data.head()

,is_sarcastic,headline,article_link
0,1,thirtysomething scientists unveil doomsday clo...,https://www.theonion.com/thirtysomething-scien...
1,0,dem rep. totally nails why congress is falling...,https://www.huffingtonpost.com/entry/donna-edw...
2,0,eat your veggies: 9 deliciously different recipes,https://www.huffingtonpost.com/entry/eat-your-...
3,1,inclement weather prevents liar from getting t...,https://local.theonion.com/inclement-weather-p...
4,1,mother comes pretty close to using word 'strea...,https://www.theonion.com/mother-comes-pretty-c...


Đưa dữ liệu headline và labels vào list tương ứng

In [ ]:
features = data['headline']
labels = data['is_sarcastic']

Loại bỏ các ký tự đặc biệt, số và các từ không cần thiết trong headline để chuyển hóa thành vector vì .fit không thực hiện khi dữ liệu là string

In [ ]:
features = features.apply(lambda s : re.sub('[^a-zA-Z]', ' ', s))

In [ ]:
stop_words = set(stopwords.words('english'))
for i in range (len(features)):
  word_tokens = word_tokenize(features[i])

  filtered_sentence = [w for w in word_tokens if not w in stop_words]

  filtered_sentence = []

  for w in word_tokens:
      if w not in stop_words:
          filtered_sentence.append(w)

Vector hóa dữ liệu bằng TF-IDF với max_feature = 5000:
- TF: tần suất xuất hiện của 1 từ trong 1 văn bản
- IDF: tần suất nghịch của 1 từ trong 1 tập văn bản
- .fit: Máy học từ vựng và idf( tần suất nghịch của 1 từ trong tập các văn bản) của training set.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfIdfVec = TfidfVectorizer(max_features = 5000)
tfIdfVec.fit(features)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=5000,
                min_df=1, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

Đưa dữ liệu headlines về matrix để training model vì hàm .fit chỉ thực hiện với dữ liệu số

In [ ]:
tf_idf_x = tfIdfVec.transform(features)

 Tiến hành chia dữ liệu train và test: theo tỉ lệ 90% training và 10% testing vì khi thực hiện theo tỉ lệ 80/20 hoặc 70/30 thì độ chính xác giảm

In [ ]:
from sklearn.model_selection import train_test_split
features_train, features_test, labels_train, labels_test = train_test_split(tf_idf_x.toarray(), labels, test_size = .1, random_state = 0)

# Model

Thêm thư viện sklearn gồm :
- Các model sử dụng : LinearSVC, Gaussian Naive Bayes, Logistic Regresstion, Random Forest
- train_test_split: để chia dữ liệu trong dataset thành train set vầ test set

In [ ]:
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

Thực hiện training các model và đánh giá performance của các model thông qua F1 Score

In [ ]:
from sklearn.metrics import f1_score

Model Linear Support Vector Classifier

In [ ]:
lsvc = LinearSVC()
lsvc.fit(features_train, labels_train)
y_pred=lsvc.predict(features_test)
f1 = f1_score(labels_test, y_pred, average='macro')
print("Test accuracy: ",lsvc.score(features_test, labels_test))
print("F1 score: ",f1)

Test accuracy:  0.8424178895877009
F1 score:  0.842135716647039


Model Gaussian Naive Bayes

In [ ]:
gnb = GaussianNB()
gnb.fit(features_train, labels_train)
y_pred=gnb.predict(features_test)
f1 = f1_score(labels_test, y_pred, average='macro')
print("Test accuracy: ",gnb.score(features_test, labels_test))
print("F1 score: ",f1)

Test accuracy:  0.7578616352201258
F1 score:  0.7574968610625399


Model Logistic Regression

In [ ]:
lr = LogisticRegression()
lr.fit(features_train, labels_train)
y_pred=lr.predict(features_test)
f1 = f1_score(labels_test, y_pred, average='macro')
print("Test accuracy: ",lr.score(features_test, labels_test))
print("F1 score: ",f1)

Test accuracy:  0.8452131376659678
F1 score:  0.8449540224192296


Model Random Forest Classifier

In [ ]:
rfc = RandomForestClassifier(n_estimators = 10, random_state = 0)
rfc.fit(features_train, labels_train)
y_pred=rfc.predict(features_test)
f1 = f1_score(labels_test, y_pred, average='macro')
print("Test accuracy: ",rfc.score(features_test, labels_test))
print("F1 score: ",f1)

Test accuracy:  0.7917540181691125
F1 score:  0.791225496027812


=> Sau khi training và đánh giá model thông qua F1 score, ta thấy Model Logistic Regression đạt được độ chính xác cao nhất

# Crawl data

Crawl data từ 2 trang web: https://www.theonion.com (tiêu đề châm biếm) và https://www.huffingtonpost.co.uk (tiêu đề không châm biếm)
- Thư viện Request: dùng đề gửi yêu cầu đến url. 
- Thư viện BeautifulSoup: dùng để lấy dữ liệu từ html(cụ thể ở đây là headline và article_link của từng bài viết).

In [ ]:
import requests
from bs4 import BeautifulSoup

Thu thập từ dữ liệu từ hai mục News in brief và News in Photos của trang https://www.theonion.com và đưa dữ liệu vào file dataset.json

In [ ]:
url = "https://www.theonion.com/c/news-in-brief"
url_1 = url
with open('dataset.json', 'a') as file:

  for i in range(26):
  
    r1 = requests.get(url_1)
    coverpage = r1.content
    soup1 = BeautifulSoup(coverpage, 'lxml')
    soup2 = BeautifulSoup(coverpage, 'html5lib')
    coverpage_news = soup1.find_all('h2', class_= "sc-759qgu-0 cYlVdn cw4lnv-6 eXwNRE")
    coverpage_news_1 = soup2.find_all('a', class_='sc-1out364-0 hMndXN js_link')
  
    t = 12
    while(t <=50):
      a = coverpage_news_1[t]['href']
      b = coverpage_news_1[t].get_text()
   
    
      file.write('{"is_sarcastic": 1, "headline": "')
      file.write(b)
      file.write('", "article_link": "')
      file.write(a)
      file.write('"}\n')

      t += 2
    url_1 = url + coverpage_news_1[51]['href']
  file.close()

In [ ]:
url = "https://www.theonion.com/c/news-in-photos"
url_1 = url
with open('dataset.json', 'a') as file:

  for i in range(26):
  
    r1 = requests.get(url_1)
    coverpage = r1.content
    soup1 = BeautifulSoup(coverpage, 'lxml')
    soup2 = BeautifulSoup(coverpage, 'html5lib')
    coverpage_news = soup1.find_all('h2', class_= "sc-759qgu-0 cYlVdn cw4lnv-6 eXwNRE")
    coverpage_news_1 = soup2.find_all('a', class_='sc-1out364-0 hMndXN js_link')
  
    t = 12
    while(t <=50):
      a = coverpage_news_1[t]['href']
      b = coverpage_news_1[t].get_text()
   
    
      file.write('{"is_sarcastic": 1, "headline": "')
      file.write(b)
      file.write('", "article_link": "')
      file.write(a)
      file.write('"}\n')

      t += 2
    url_1 = url + coverpage_news_1[51]['href']
  file.close()

Thu thập dữ liệu từ hai mục News của trang https://www.huffingtonpost.co.uk và đưa dữ liệu vào file dataset.json

In [ ]:
from urllib.request import Request , urlopen 
from bs4 import BeautifulSoup
Data = []
with open('dataset.json', 'a') as file:
  for i in range(1,35):
    url1 = 'https://www.huffingtonpost.co.uk/news/'+ str(i) +'/?guccounter=2' 
    req = Request(url1, headers={'User-Agent': 'XYZ/3.0'})
    page = urlopen(req,timeout=10) 
    soup = BeautifulSoup(page,"html.parser") 
    headlines = soup.find_all('a',attrs={'class':'card__link yr-card-headline'}) 

    for i in range(len(headlines)):
      headline = headlines[i].text 
      if headline not in Data:
        Data.append(headline)
        link = 'https://www.huffingtonpost.co.uk' + headlines[i]['href'] 

        file.write('{"is_sarcastic": 0, "headline": "')
        file.write(headline)
        file.write('", "article_link": "')
        file.write(link)
        file.write('"}\n')
  file.close()

# Thực hiện dự đoán trên dataset mới

Đọc dữ liệu của dataset mới

In [ ]:
data1 = pd.read_json("dataset.json", lines = True, encoding='utf-8' )
data1.head()

,is_sarcastic,headline,article_link
0,1,Nancy Pelosi Calls Jamaal Bowman To Scold Him ...,https://politics.theonion.com/nancy-pelosi-cal...
1,1,"Gamers, We Just Spent 4 Days Trapped In A Roll...",https://ogn.theonion.com/gamers-we-just-spent-...
2,1,Defiant Florida Officials Announce They Will I...,https://www.theonion.com/defiant-florida-offic...
3,1,"Employee Leaves Performance Review With Clear,...",https://www.theonion.com/employee-leaves-perfo...
4,1,Officials Warn Defunding Police Could Lead To ...,https://www.theonion.com/officials-warn-defund...


Đưa dữ liệu headlines và labels vào các biến tương ứng

In [ ]:
features_val = data1['headline']
labels_val = data1['is_sarcastic']

Feature engineering

Loại bỏ các ký tự đặc biệt và số trong headlines

In [ ]:
# Relacing special symbols and digits in headline column
# re stands for Regular Expression
features = features.apply(lambda s : re.sub('[^a-zA-Z]', ' ', s))

Thực hiện tách từ và loại bỏ các stopwords trong headlines

In [ ]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

stop_words = set(stopwords.words('english'))
for i in range (len(features_val)):
  word_tokens = word_tokenize(features_val[i])

  filtered_sentence = [w for w in word_tokens if not w in stop_words]

  filtered_sentence = []

  for w in word_tokens:
      if w not in stop_words:
          filtered_sentence.append(w)

Vector hóa dữ liệu

In [ ]:
tv1 = tfIdfVec
#features_val = list(features_val)
features_val = tv1.transform(features_val).toarray()

# Tiến hành dự đoán trên model có độ chính xác cao nhất(Logistic Regression)

- Thông qua kết quả dự đoán với 2000 headlines mới, ta thấy performance giảm khi sử dụng model được train từ dữ liệu cũ (84,5% -> 77,4%). Lý do performance giảm vì trong dataset mới có các từ chưa từng xuất hiện trong train set và các từ mới xuất hiện hiện nay như(coronavirus,...)



In [ ]:
print("Val accuracy: ",lr.score(features_val, labels_val))

Val accuracy:  0.7743785850860421


# Chương trình Sarcasm Detection

Xử lý dữ liệu mới nhập vào

In [ ]:
ch=(str(input()))
ch = re.sub(r'[^a-zA-Z]',' ',ch)
ch_tokens = word_tokenize(ch)
ch_sentence = [w for w in ch_tokens if not w in stop_words]
ch = " ".join(ch)

WikiLeaks Founder Julian Assange Faces New Indictment In U.S.


Đưa dữ liệu về ma trận để dự đoán trên model logistic regression

In [ ]:
ch_val = tv1.transform([ch]).toarray()

In [ ]:
ch_pred=lr.predict(ch_val)

Sau khi tiến hành dự đoán 10 tiêu đề bất kỳ kết quả dự đoán: 4 đúng -  6 sai, lý do kết quả dự đoán sai vì có nhiều words chưa từng xuất hiện khi được train.

In [ ]:
if(ch_pred==1):
  print("sarcasm")
if(ch_pred==0):
  print("not sarcasm")

not sarcasm


# Nhận xét của nhóm về bài toán

Sau khi hoàn thành bài toán sarcasm detection, nhóm em nhận thấy bài toán này khá hay và dữ liệu text và còn nhiều vấn đề cần giải quyết khi đưa dữ liệu mới vào để dự đoán vd như: cách giải quyết các từ mới xuất hiện, các tên riêng,.... 